In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import lightgbm as lgb
import plotly.express as px
from itertools import combinations
import os
import gc
import sys

import warnings
warnings.filterwarnings("ignore")

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [ ]:
# Load the optiver files

train_df = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")
revealed_targets_df = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv')
test_df = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv')
sample_submission_df = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv')

In [ ]:
# Overview of the optiver data

train_df.sort_values(by=['stock_id','date_id'],inplace=True)
train_df.loc[train_df['stock_id']==0]

train_df

In [ ]:
def feature_engineering(df):
    
    # Removing data that's not in the actual test data in the API
    cols = [c for c in df.columns if c not in ['row_id', 'time_id', 'target']]
    df = df[cols]
    
    # Here we calculate spread differentials and spread ratios
    df['bid_minus_ask_size'] = df['bid_size'] - df['ask_size']
    df['bid_ask_size_ratio'] = df['bid_size'] / df['ask_size']
    df['bid_ask_price_ratio'] = df['bid_size'] / df['ask_size']
    df['bid_minus_ask_spread'] = df['ask_price'] - df['bid_price']
    
    # Next, we calculate median, matched sizes, bid and ask sizes
    stock_median_ask_size = df.groupby(['stock_id'])['ask_size'].median()
    stock_median_bid_size = df.groupby(['stock_id'])['bid_size'].median()
    stock_median_imbalance_size = df.groupby(['stock_id'])['imbalance_size'].median()
    
    day_median_ask_size = df.groupby(['date_id'])['ask_size'].median()
    day_median_bid_size = df.groupby(['date_id'])['bid_size'].median()
    day_median_imbalance_size = df.groupby(['date_id'])['imbalance_size'].median()
    
    day_stock_ask_median_ask_size = df.groupby(['date_id', 'stock_id'])['ask_size'].median()
    day_stock_bid_median_ask_size = df.groupby(['date_id', 'stock_id'])['bid_size'].median()
    day_stock_imbalance_median_ask_size = df.groupby(['date_id', 'stock_id'])['imbalance_size'].median()
    
    df = df.merge(stock_median_ask_size, how='left', left_on='stock_id', right_index=True, suffixes=('', '_stock_median'))
    df = df.merge(stock_median_bid_size, how='left', left_on='stock_id', right_index=True, suffixes=('', '_stock_median'))
    df = df.merge(stock_median_imbalance_size, how='left', left_on='stock_id', right_index=True, suffixes=('', '_stock_median'))
    
    # Lag features
    lag_features = ['bid_size', 'ask_size', 'imbalance_size']
    for feature in lag_features:
        df[f'{feature}_lag_1'] = df.groupby(['stock_id'])[feature].shift(1)
    
    # Rolling statistics
    rolling_features = ['bid_size', 'ask_size', 'imbalance_size']
    window_sizes = [5, 10, 20]
    for feature in rolling_features:
        for window_size in window_sizes:
            df[f'{feature}_rolling_mean_{window_size}'] = df.groupby(['stock_id'])[feature].transform(lambda x: x.rolling(window=window_size).mean())
            df[f'{feature}_rolling_std_{window_size}'] = df.groupby(['stock_id'])[feature].transform(lambda x: x.rolling(window=window_size).std())
    
    # Advanced time series features
    df['return'] = df['wap'].pct_change()
    df['log_return'] = np.log(1 + df['return'])
    df['cumulative_return'] = (1 + df['return']).cumprod()
    
    # Autocorrelation features
    autocorr_features = ['bid_size', 'ask_size', 'imbalance_size']
    for feature in autocorr_features:
        df[f'{feature}_autocorr_1'] = df.groupby(['stock_id'])[feature].transform(lambda x: x.autocorr(1))
        df[f'{feature}_autocorr_3'] = df.groupby(['stock_id'])[feature].transform(lambda x: x.autocorr(3))
    
    # Drop unnecessary columns
    df.drop(columns=['date_id'], axis=1, inplace=True)
    
    gc.collect()
    
    return df

In [ ]:
y = train_df['target'].values
X = feature_engineering(train_df)

In [ ]:
X.tail(10)

In [ ]:
X.columns

In [ ]:
model = lgb.LGBMRegressor(objective='mae', 
                      n_estimators=600, 
                      random_state=51, num_threads=4, num_leaves=35)

In [ ]:
model.fit(X, y)

In [ ]:
len(model.feature_importances_)

In [ ]:
feature_imp = pd.Series(model.feature_importances_, index=X.columns).sort_values()
print('Columns with poor contribution', feature_imp[feature_imp<15].index)
fig = px.bar(x=feature_imp, y=feature_imp.index, orientation='h')
fig.show()

In [ ]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [ ]:
counter = 0
Y_actual = pd.DataFrame()
for (test, revealed_targets, sample_prediction) in iter_test:
    x = feature_engineering(test)
    x.drop("currently_scored", axis=1, inplace=True)
    sample_prediction['target'] = model.predict(x)
    env.predict(sample_prediction)
    #concat
    Y_actual = pd.concat( [Y_actual, revealed_targets])
    counter += 1